In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

fifa_df = pd.read_csv('../preprocessing/clean_data.csv')
fifa_df.head()

,ID,name,full_name,club,special,age,league,height_cm,weight_kg,body_type,...,prefers_cb,prefers_lb,prefers_lwb,prefers_ls,prefers_lf,prefers_lam,prefers_lcm,prefers_ldm,prefers_lcb,prefers_gk
0,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Real Madrid CF,2228,32,Spanish Primera División,185.0,80.0,Lean,...,False,False,False,False,False,False,False,False,False,False
1,158023,L. Messi,Lionel Messi,FC Barcelona,2158,30,Spanish Primera División,170.0,72.0,Lean,...,False,False,False,False,False,False,False,False,False,False
2,190871,Neymar,Neymar da Silva Santos Jr.,Paris Saint-Germain,2100,25,French Ligue 1,175.0,68.0,Lean,...,False,False,False,False,False,False,False,False,False,False
3,176580,L. Suárez,Luis Suárez,FC Barcelona,2291,30,Spanish Primera División,182.0,86.0,Normal,...,False,False,False,False,False,False,False,False,False,False
4,167495,M. Neuer,Manuel Neuer,FC Bayern Munich,1493,31,German Bundesliga,193.0,92.0,Normal,...,False,False,False,False,False,False,False,False,False,True


#### Separação das variáveis. As categorical_name são as categóricas não booleanas.

In [2]:
numerical_variables = ['eur_value', 'eur_wage', 'eur_release_clause','age', 'height_cm', 'weight_kg', 'ID', 'special', 'overall', 'potential', 'pac', 'sho', 'pas', 'dri', 'def', 'phy', 'international_reputation', 'skill_moves', 'weak_foot', 'crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina', 'strength',
'long_shots', 'aggression', 'interceptions', 'positioning', 'vision', 'penalties', 'composure', 'marking', 'standing_tackle', 'sliding_tackle', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes', 'rs', 'rw', 'rf', 'ram', 'rcm', 'rm', 'rdm', 'rcb', 'rb', 'rwb', 'st', 'lw', 'cf', 'cam', 'cm', 'lm', 'cdm', 'cb', 'lb', 'lwb', 'ls', 'lf', 'lam', 'lcm', 'ldm', 'lcb', 'gk']

categorical_variables = [c for c in fifa_df.columns if c not in numerical_variables]
numerical_variables = [c for c in numerical_variables if c in fifa_df.columns]
categorical_name_variables = ['body_type', 'work_rate_att', 'work_rate_def', 'preferred_foot', 'nationality', 'league', 'special']

#### Definição da posição do jogador como a que ele tem melhor desempenho

In [3]:
original_positions = 'rs, rw, rf, ram, rcm, rm, rdm, rcb, rb, rwb, st, lw, cf, cam, cm, lm, cdm, cb, lb, lwb, ls, lf, lam, lcm, ldm, lcb, gk'
original_positions = original_positions.replace(' ', '').split(',')
fifa_df['best_pos'] = fifa_df[original_positions].idxmax(axis='columns')

#### Retirada de variáveis consideradas pouco relevantes para o processo

In [4]:
to_drop = [v for v in numerical_variables if 'gk' in v or 'prefers' in v]
to_drop =['name', 'full_name', 'ID', 'eur_release_clause', 'club', 'league']+to_drop
to_drop = to_drop+[v for v in categorical_variables if 'gk' in v or 'prefers' in v]
to_drop = to_drop + [c for c in categorical_variables if c not in categorical_name_variables]

numerical_variables = [v for v in numerical_variables if v not in to_drop]
categorical_variables = [v for v in categorical_variables if v not in to_drop]

fifa_df.drop(to_drop, axis='columns', inplace=True)
to_drop

['name',
 'full_name',
 'ID',
 'eur_release_clause',
 'club',
 'league',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes',
 'gk',
 'gk_flat_kick_trait',
 'gk_long_throw_trait',
 'gk_up_for_corners_trait',
 'prefers_rs',
 'prefers_rw',
 'prefers_rf',
 'prefers_ram',
 'prefers_rcm',
 'prefers_rm',
 'prefers_rdm',
 'prefers_rcb',
 'prefers_rb',
 'prefers_rwb',
 'prefers_st',
 'prefers_lw',
 'prefers_cf',
 'prefers_cam',
 'prefers_cm',
 'prefers_lm',
 'prefers_cdm',
 'prefers_cb',
 'prefers_lb',
 'prefers_lwb',
 'prefers_ls',
 'prefers_lf',
 'prefers_lam',
 'prefers_lcm',
 'prefers_ldm',
 'prefers_lcb',
 'prefers_gk',
 'name',
 'full_name',
 'club',
 '1_on_1_rush_trait',
 'acrobatic_clearance_trait',
 'argues_with_officials_trait',
 'avoids_using_weaker_foot_trait',
 'backs_into_player_trait',
 'bicycle_kicks_trait',
 'cautious_with_crosses_trait',
 'chip_shot_trait',
 'chipped_penalty_trait',
 'comes_for_crosses_trait',
 'corner_specialist_trait',
 'diver_tra

In [5]:
# Obtenção de todos os atacantes
strikers_position = ['st', 'cf', 'lw', 'rw']
strikers_df = fifa_df[fifa_df.best_pos.isin(strikers_position)]
strikers_df.describe()

,special,age,height_cm,weight_kg,eur_value,eur_wage,overall,potential,pac,sho,...,cdm,cb,lb,lwb,ls,lf,lam,lcm,ldm,lcb
count,3108.000000,3108.000000,3108.000000,3108.000000,3.108000e+03,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,...,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000
mean,1691.378057,24.048584,176.073037,70.474903,2.933330e+06,12462.998713,66.619048,72.253539,77.203024,61.152510,...,50.348134,44.586551,51.453024,54.109717,62.788610,65.238417,65.170849,60.175676,50.348134,44.586551
std,173.067351,4.049169,5.439382,5.471083,6.278811e+06,22788.025196,6.712472,6.072733,7.781141,7.967259,...,7.760648,8.379049,7.286454,7.017189,6.677261,6.827695,6.866474,7.149876,7.760648,8.379049
min,1185.000000,16.000000,155.000000,49.000000,0.000000e+00,0.000000,46.000000,54.000000,42.000000,36.000000,...,28.000000,25.000000,30.000000,32.000000,43.000000,45.000000,45.000000,39.000000,28.000000,25.000000
25%,1569.000000,21.000000,172.000000,67.000000,4.250000e+05,2000.000000,62.000000,68.000000,72.000000,56.000000,...,45.000000,38.000000,46.000000,49.000000,58.000000,61.000000,61.000000,55.000000,45.000000,38.000000
50%,1694.000000,24.000000,176.000000,70.000000,8.750000e+05,5000.000000,67.000000,72.000000,77.000000,61.000000,...,50.000000,43.000000,51.000000,54.000000,63.000000,65.000000,65.000000,60.000000,50.000000,43.000000
75%,1812.000000,27.000000,180.000000,74.000000,2.700000e+06,13000.000000,71.000000,76.000000,83.000000,67.000000,...,56.000000,50.000000,56.000000,59.000000,67.000000,70.000000,70.000000,65.000000,56.000000,50.000000
max,2265.000000,38.000000,196.000000,90.000000,1.230000e+08,295000.000000,92.000000,94.000000,96.000000,87.000000,...,79.000000,76.000000,80.000000,80.000000,85.000000,88.000000,88.000000,81.000000,79.000000,76.000000


In [6]:
strikers_df.head()

,special,age,height_cm,weight_kg,body_type,nationality,eur_value,eur_wage,overall,potential,...,cb,lb,lwb,ls,lf,lam,lcm,ldm,lcb,best_pos
2,2100,25,175.0,68.0,Lean,Brazil,123000000.0,280000.0,92,94,...,46.0,59.0,64.0,84.0,88.0,88.0,79.0,59.0,46.0,rw
7,2096,26,173.0,76.0,Normal,Belgium,90500000.0,295000.0,90,91,...,47.0,59.0,64.0,82.0,87.0,88.0,81.0,61.0,47.0,rw
13,2181,28,169.0,62.0,Normal,Chile,67500000.0,265000.0,89,89,...,56.0,62.0,66.0,83.0,85.0,85.0,79.0,64.0,56.0,rw
19,2063,23,177.0,73.0,Normal,Argentina,79000000.0,215000.0,88,93,...,43.0,55.0,60.0,83.0,86.0,86.0,78.0,55.0,43.0,rw
21,2104,26,174.0,72.0,Lean,France,75000000.0,150000.0,88,91,...,48.0,56.0,60.0,85.0,85.0,84.0,76.0,57.0,48.0,rw


In [7]:
# Retirada de colunas com alta correlação

correlation_threshold = 0.8
# Identificando colunas altamente relacionadas
correlation_matrix = strikers_df.corr().abs()
# Seleciona o triângulo superior da matriz de correlação
upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool))
# Seleciona colunas a serem removidas
to_drop = [column for column in upper.columns if any(upper[column] > correlation_threshold)]
#to_drop.remove('overall')
to_drop.remove('overall')

In [8]:
# Removendo entradas com altas correlação
#to_drop = 'eur_release_clause'
strikers_df = strikers_df.drop(to_drop, axis='columns')
numerical_variables = [var for var in numerical_variables if var not in to_drop]
categorical_variables = [var for var in categorical_variables if var not in to_drop]
#strikers_df.head()

In [9]:
# Seleção dos jogadores das três nacionalidades previamente definidas
countries = ['Brazil', 'Italy', 'Germany']
strikers_df = strikers_df[strikers_df.nationality.isin(countries)]
strikers_df.nationality.value_counts()

Germany    189
Brazil     147
Italy      106
Name: nationality, dtype: int64

In [10]:
strikers_df.as_matrix().shape

(442, 30)

In [11]:
#Ordenando os jogadores pelo atributo 'overall'
strikers_df.sort_values(by='overall', ascending=False, inplace=True)
strikers_df.drop(['overall', 'best_pos'], axis=1, inplace=True)
numerical_variables.remove('overall')

In [12]:
# discretizando as variáveis numéricas em 10 invervalos
num_buckets = 10
for v in numerical_variables:
    strikers_df[v] = pd.cut(strikers_df[v], num_buckets).apply(lambda x: v+'_'+str(x)).astype('category')
    
for v in categorical_variables:
    strikers_df[v] = strikers_df[v].apply(lambda x: v+'_'+str(x))

strikers_df.head()

,special,age,height_cm,weight_kg,body_type,nationality,eur_value,potential,pac,def,...,volleys,curve,free_kick_accuracy,agility,balance,jumping,stamina,aggression,penalties,composure
2,"special_(2073.4, 2169.2]","age_(24.6, 26.5]","height_cm_(171.8, 175.0]","weight_kg_(66.4, 69.2]",body_type_Lean,nationality_Brazil,"eur_value_(110706000.0, 123000000.0]","potential_(90.0, 94.0]","pac_(88.9, 94.0]","def_(27.2, 33.3]",...,"volleys_(82.3, 88.0]","curve_(78.4, 84.2]","free_kick_accuracy_(82.9, 89.0]","agility_(91.2, 96.0]","balance_(81.3, 86.2]","jumping_(60.5, 66.4]","stamina_(72.0, 78.0]","aggression_(51.5, 57.2]","penalties_(80.5, 87.0]","composure_(86.4, 92.0]"
55,"special_(2073.4, 2169.2]","age_(26.5, 28.4]","height_cm_(178.2, 181.4]","weight_kg_(74.8, 77.6]",body_type_Lean,nationality_Germany,"eur_value_(36942000.0, 49236000.0]","potential_(82.0, 86.0]","pac_(83.8, 88.9]","def_(33.3, 39.4]",...,"volleys_(82.3, 88.0]","curve_(84.2, 90.0]","free_kick_accuracy_(82.9, 89.0]","agility_(81.6, 86.4]","balance_(76.4, 81.3]","jumping_(66.4, 72.3]","stamina_(66.0, 72.0]","aggression_(45.8, 51.5]","penalties_(80.5, 87.0]","composure_(80.8, 86.4]"
73,"special_(1881.8, 1977.6]","age_(24.6, 26.5]","height_cm_(162.2, 165.4]","weight_kg_(57.972, 60.8]",body_type_Normal,nationality_Italy,"eur_value_(36942000.0, 49236000.0]","potential_(82.0, 86.0]","pac_(88.9, 94.0]","def_(21.1, 27.2]",...,"volleys_(70.9, 76.6]","curve_(78.4, 84.2]","free_kick_accuracy_(76.8, 82.9]","agility_(91.2, 96.0]","balance_(91.1, 96.0]","jumping_(48.7, 54.6]","stamina_(72.0, 78.0]","aggression_(28.7, 34.4]","penalties_(54.5, 61.0]","composure_(75.2, 80.8]"
115,"special_(1977.6, 2073.4]","age_(26.5, 28.4]","height_cm_(171.8, 175.0]","weight_kg_(74.8, 77.6]",body_type_Lean,nationality_Brazil,"eur_value_(24648000.0, 36942000.0]","potential_(82.0, 86.0]","pac_(83.8, 88.9]","def_(45.5, 51.6]",...,"volleys_(53.8, 59.5]","curve_(78.4, 84.2]","free_kick_accuracy_(82.9, 89.0]","agility_(86.4, 91.2]","balance_(76.4, 81.3]","jumping_(42.8, 48.7]","stamina_(84.0, 90.0]","aggression_(40.1, 45.8]","penalties_(67.5, 74.0]","composure_(75.2, 80.8]"
101,"special_(1977.6, 2073.4]","age_(22.7, 24.6]","height_cm_(184.6, 187.8]","weight_kg_(69.2, 72.0]",body_type_Lean,nationality_Germany,"eur_value_(36942000.0, 49236000.0]","potential_(86.0, 90.0]","pac_(73.6, 78.7]","def_(33.3, 39.4]",...,"volleys_(82.3, 88.0]","curve_(72.6, 78.4]","free_kick_accuracy_(70.7, 76.8]","agility_(76.8, 81.6]","balance_(71.5, 76.4]","jumping_(72.3, 78.2]","stamina_(66.0, 72.0]","aggression_(45.8, 51.5]","penalties_(61.0, 67.5]","composure_(80.8, 86.4]"


In [13]:
# separação dos jogadores por nacionalidade, obtendo somente os 50 melhores de cada nacionalidade
country2strikers = {}
for country in countries:
    country2strikers[country] = strikers_df[strikers_df.nationality == 'nationality_'+country].head(100).drop('nationality', axis=1)

In [14]:
for country in countries:
    country2strikers[country].to_csv('atacantes/'+country+'.csv', index=False)

### obtenção das regras de associação
- Feito no RapidMiner
- Extração de padrões com FPGrowth, suporte mínimo de 0.8, mínimo de 1 itens
- Regras de associação com confiança mínima de 0.8

In [32]:
rules = {}
for country in countries:
    rules[country] = pd.read_csv('regras/'+country+'.csv', sep='|').drop('id', axis=1)

In [37]:
# Apagando regras com o suporte menor que 0.5
for country in countries:
    df = rules[country]
    rules[country] = df[df.suporte>=0.5].sort_values('premissa')

### Visualização das regras de associação dos atacantes de cada país

In [40]:
print("Quantidade de regras de associação por país")
for country in countries:
    print('{}: {}'.format(country, len(rules[country])))

Quantidade de regras de associação por país
Brazil: 12
Italy: 18
Germany: 56


In [41]:
rules['Brazil']

,premissa,conclusao,suporte,confiança
65,"eur_value_(-62940.0, 12354000.0]","international_reputation_(0.996, 1.4]",0.84,0.943820
108,"eur_value_(-62940.0, 12354000.0], preferred_fo...","international_reputation_(0.996, 1.4]",0.56,1.000000
59,"eur_value_(-62940.0, 12354000.0], work_rate_de...","international_reputation_(0.996, 1.4]",0.59,0.936508
99,"international_reputation_(0.996, 1.4]","eur_value_(-62940.0, 12354000.0]",0.84,0.976744
103,"international_reputation_(0.996, 1.4], preferr...","eur_value_(-62940.0, 12354000.0]",0.56,0.982456
104,"international_reputation_(0.996, 1.4], work_ra...","eur_value_(-62940.0, 12354000.0]",0.59,0.983333
18,preferred_foot_Right,"eur_value_(-62940.0, 12354000.0]",0.56,0.875000
19,preferred_foot_Right,"eur_value_(-62940.0, 12354000.0], internationa...",0.56,0.875000
25,preferred_foot_Right,"international_reputation_(0.996, 1.4]",0.57,0.890625
11,work_rate_def_Medium,"eur_value_(-62940.0, 12354000.0], internationa...",0.59,0.855072


In [38]:
rules['Germany']

,premissa,conclusao,suporte,confiança
110,"eur_value_(-62940.0, 12354000.0]","international_reputation_(0.996, 1.4]",0.80,0.898876
43,"eur_value_(-62940.0, 12354000.0]",work_rate_def_Medium,0.76,0.853933
88,"eur_value_(-62940.0, 12354000.0], internationa...",work_rate_def_Medium,0.71,0.887500
102,"eur_value_(-62940.0, 12354000.0], internationa...",work_rate_def_Medium,0.50,0.892857
98,"eur_value_(-62940.0, 12354000.0], internationa...",work_rate_def_Medium,0.57,0.890625
103,"eur_value_(-62940.0, 12354000.0], internationa...",work_rate_def_Medium,0.50,0.892857
46,"eur_value_(-62940.0, 12354000.0], preferred_fo...",work_rate_def_Medium,0.54,0.857143
91,"eur_value_(-62940.0, 12354000.0], preferred_fo...","international_reputation_(0.996, 1.4]",0.56,0.888889
45,"eur_value_(-62940.0, 12354000.0], skill_moves_...",work_rate_def_Medium,0.60,0.857143
131,"eur_value_(-62940.0, 12354000.0], skill_moves_...","international_reputation_(0.996, 1.4]",0.64,0.914286


In [39]:
rules['Italy']

,premissa,conclusao,suporte,confiança
30,"eur_value_(-62940.0, 12354000.0]","international_reputation_(0.996, 1.4]",0.80,0.879121
108,"eur_value_(-62940.0, 12354000.0], preferred_fo...","international_reputation_(0.996, 1.4]",0.56,1.000000
34,"eur_value_(-62940.0, 12354000.0], skill_moves_...","international_reputation_(0.996, 1.4]",0.61,0.884058
46,"eur_value_(-62940.0, 12354000.0], work_rate_de...","international_reputation_(0.996, 1.4]",0.60,0.909091
66,"international_reputation_(0.996, 1.4]","eur_value_(-62940.0, 12354000.0]",0.80,1.000000
99,"international_reputation_(0.996, 1.4]","eur_value_(-62940.0, 12354000.0]",0.84,0.976744
103,"international_reputation_(0.996, 1.4], preferr...","eur_value_(-62940.0, 12354000.0]",0.56,0.982456
68,"international_reputation_(0.996, 1.4], skill_m...","eur_value_(-62940.0, 12354000.0]",0.61,1.000000
67,"international_reputation_(0.996, 1.4], work_ra...","eur_value_(-62940.0, 12354000.0]",0.60,1.000000
104,"international_reputation_(0.996, 1.4], work_ra...","eur_value_(-62940.0, 12354000.0]",0.59,0.983333


In [56]:
rules_str = {}
for country in countries:
    rules_str[country] = (rules[country].premissa +"->" +rules[country].conclusao).values

In [62]:
from  pprint import pprint

In [63]:
## Brasil e Italia
c = []
for rule in rules_str['Brazil']:
    if rule in rules_str['Italy']:
        c.append(rule)
pprint(c), len(c)

['eur_value_(-62940.0, 12354000.0]->international_reputation_(0.996, 1.4]',
 'eur_value_(-62940.0, 12354000.0], '
 'preferred_foot_Right->international_reputation_(0.996, 1.4]',
 'eur_value_(-62940.0, 12354000.0], '
 'work_rate_def_Medium->international_reputation_(0.996, 1.4]',
 'international_reputation_(0.996, 1.4]->eur_value_(-62940.0, 12354000.0]',
 'international_reputation_(0.996, 1.4], '
 'preferred_foot_Right->eur_value_(-62940.0, 12354000.0]',
 'international_reputation_(0.996, 1.4], '
 'work_rate_def_Medium->eur_value_(-62940.0, 12354000.0]',
 'preferred_foot_Right->eur_value_(-62940.0, 12354000.0]',
 'work_rate_def_Medium->eur_value_(-62940.0, 12354000.0]']


(None, 8)

In [64]:
## Brasil e Alemanha
c = []
for rule in rules_str['Brazil']:
    if rule in rules_str['Germany']:
        c.append(rule)
pprint(c), len(c)

['eur_value_(-62940.0, 12354000.0]->international_reputation_(0.996, 1.4]',
 'eur_value_(-62940.0, 12354000.0], '
 'preferred_foot_Right->international_reputation_(0.996, 1.4]',
 'eur_value_(-62940.0, 12354000.0], '
 'work_rate_def_Medium->international_reputation_(0.996, 1.4]',
 'international_reputation_(0.996, 1.4]->eur_value_(-62940.0, 12354000.0]',
 'international_reputation_(0.996, 1.4], '
 'preferred_foot_Right->eur_value_(-62940.0, 12354000.0]',
 'preferred_foot_Right->eur_value_(-62940.0, 12354000.0]',
 'work_rate_def_Medium->eur_value_(-62940.0, 12354000.0], '
 'international_reputation_(0.996, 1.4]',
 'work_rate_def_Medium->international_reputation_(0.996, 1.4]',
 'work_rate_def_Medium->eur_value_(-62940.0, 12354000.0]']


(None, 9)

In [65]:
# Italia e Alemanha
c = []
for rule in rules_str['Germany']:
    if rule in rules_str['Italy']:
        c.append(rule)
pprint(c), len(c)

['eur_value_(-62940.0, 12354000.0]->international_reputation_(0.996, 1.4]',
 'eur_value_(-62940.0, 12354000.0], '
 'preferred_foot_Right->international_reputation_(0.996, 1.4]',
 'eur_value_(-62940.0, 12354000.0], skill_moves_(2.9, '
 '3.2]->international_reputation_(0.996, 1.4]',
 'eur_value_(-62940.0, 12354000.0], '
 'work_rate_def_Medium->international_reputation_(0.996, 1.4]',
 'international_reputation_(0.996, 1.4]->eur_value_(-62940.0, 12354000.0]',
 'international_reputation_(0.996, 1.4], '
 'preferred_foot_Right->eur_value_(-62940.0, 12354000.0]',
 'international_reputation_(0.996, 1.4], skill_moves_(2.9, '
 '3.2]->eur_value_(-62940.0, 12354000.0]',
 'preferred_foot_Right->eur_value_(-62940.0, 12354000.0]',
 'skill_moves_(2.9, 3.2]->eur_value_(-62940.0, 12354000.0], '
 'international_reputation_(0.996, 1.4]',
 'skill_moves_(2.9, 3.2]->international_reputation_(0.996, 1.4]',
 'skill_moves_(2.9, 3.2]->eur_value_(-62940.0, 12354000.0]',
 'weak_foot_(2.6, 3.0]->eur_value_(-62940.0,

(None, 14)